In [1]:
import sys
sys.path.insert(0, '/home/leandro/math/sage/sage_tvars')
from tvars import *
import random
PS=ProjectiveSpace(1,QQ)
BR = QQ;

In [5]:
def p_divisor_gen(p, bound):
    # generate a p-divisor of type (1,p,q) and dimension 3 
    #bound = 5
    n=2
    q = p+1;
    v0= vector([random.choice(range(-bound, bound)), random.choice(range(-bound, bound))])
    v_inf= vector([random.choice(range(-bound, bound)), random.choice(range(-bound, bound))])
    while v0[0] % p == 0 or v0[1] % p == 0:
        v0= vector([random.choice(range(-bound, bound)), random.choice(range(-bound, bound))])
    v0 = (1/p)*v0
    while (v_inf[0] % q == 0) or (v_inf[1] % q == 0):
         v_inf= vector([random.choice(range(-bound, bound)), random.choice(range(-bound, bound))])
    v_inf = (1/q)*v_inf
    v1= vector([random.choice(range(-bound, bound)), random.choice(range(-bound, bound))])
    v2 = vector([0,0])
    #v2= vector([random.choice(range(-bound, bound)), random.choice(range(-bound, bound))])
    sigma = Cone([v0 + v1 + v_inf, v0 + v2 + v_inf]).rays()
    # the p-divisor
    dict = {PS(0,1) : Polyhedron(vertices=[v0],rays=sigma, base_ring=BR),
           PS(1,0) : Polyhedron(vertices=[v_inf],rays=sigma, base_ring=BR),
           PS(1,1) : Polyhedron(vertices =[v1,v2],rays=sigma, base_ring=BR)
           }
    D = PolyhedralDivisor(dict);
    if not D.is_proper():
        #print("not proper")
        raise NotImplementedError(f"The variety is not Q-Gorenstein.")
    if not D.is_Q_Gorenstein():
        #print("Not Q-G")
        raise NotImplementedError(f"The variety is not Q-Gorenstein.")
    return D

In [3]:
def p_divisor_gen3(p, bound):
    # generate a p-divisor of type (1,p,q) and dimension 4
    #bound = 5
    n=3
    q = p+1;
    v0= vector([random.choice(range(-bound, bound)), random.choice(range(-bound, bound)), random.choice(range(-bound, bound))])
    v_inf= vector([random.choice(range(-bound, bound)), random.choice(range(-bound, bound)), random.choice(range(-bound, bound))])
    while v0[0] % p == 0 or v0[1] % p == 0 or v0[2] % p == 0:
        v0= vector([random.choice(range(-bound, bound)), random.choice(range(-bound, bound)), random.choice(range(-bound, bound))])
    v0 = (1/p)*v0
    while (v_inf[0] % q == 0) or (v_inf[1] % q == 0) or (v_inf[2] % q == 0):
         v_inf= vector([random.choice(range(-bound, bound)), random.choice(range(-bound, bound)), random.choice(range(-bound, bound))])
    v_inf = (1/q)*v_inf
    v1= vector([random.choice(range(-bound, bound)), random.choice(range(-bound, bound)), random.choice(range(-bound, bound))])
    v2 = vector([0,0,0])
    v3= vector([random.choice(range(-bound, bound)), random.choice(range(-bound, bound)), random.choice(range(-bound, bound))])
    v4 = vector([0,0,0])
    sigma = Cone([v0 + v1 + v3 + v_inf, v0 + v1 + v4 + v_inf, v0 + v2 + v3 + v_inf, v0 + v2 + v4 + v_inf]).rays()
    # the p-divisor
    dict = {PS(0,1) : Polyhedron(vertices=[v0],rays=sigma, base_ring=BR),
           PS(1,0) : Polyhedron(vertices=[v_inf],rays=sigma, base_ring=BR),
           PS(1,1) : Polyhedron(vertices =[v1,v2],rays=sigma, base_ring=BR),
           PS(1,2) : Polyhedron(vertices =[v3,v4],rays=sigma, base_ring=BR)
           }
    D = PolyhedralDivisor(dict);
    if not D.is_proper():
        #print("not proper")
        raise NotImplementedError(f"The variety is not Q-Gorenstein.")
    if not D.is_Q_Gorenstein():
        #print("Not Q-G")
        raise NotImplementedError(f"The variety is not Q-Gorenstein.")
    return D

In [4]:
%%script false --no-raise-error
n=2
v0= vector([(1/p), (1/p)*2])
v_inf= vector([(1/q)*(-1), (1/q)*0])
v1= vector([-1,0])
v2 = vector([0,0])
sigma = Cone([v0 + v1 + v_inf, v0 + v2 + v_inf]).rays()
dict = {PS(0,1) : Polyhedron(vertices=[v0],rays=sigma, base_ring=BR),
       PS(1,0) : Polyhedron(vertices=[v_inf],rays=sigma, base_ring=BR),
       PS(1,1) : Polyhedron(vertices =[v1,v2],rays=sigma, base_ring=BR)
       }
D = PolyhedralDivisor(dict);
if not D.is_proper():
    raise NotImplementedError(f"The variety is not Q-Gorenstein.")
if not D.is_Q_Gorenstein():
    raise NotImplementedError(f"The variety is not Q-Gorenstein.")
D.deg

In [6]:
D = p_divisor_gen(3,15)

NameError: name 'PolyhedralDivisor' is not defined

In [34]:
def divisors(n, k):
    #dim(X) = n+1
    divs = []
    for i in range(k):
        p = 3
        #p = random.choice(range(2,50))
        q = p+1
        try:
            if n == 3:
                D = p_divisor_gen3(p,25)
            else:
                D = p_divisor_gen(p, 25)
        except IndexError as ie:
            print("index trouble")
            continue
        # the variety should always be Q-Gorenstein but apparently sometimes isn't 
        except NotImplementedError as e:
            print("Not Q-G.")
            continue
        if D.multiplicity_of_slice(PS(0,1)) != p:
            continue
        if D.multiplicity_of_slice(PS(1,0)) != q:
            continue
        divs.append(D)
    return divs

In [6]:
def multiplicities(D, y1,y2):
    n = D.tail.dim() + 1
    C1 = D.degeneration(y1)
    w = vector(QQ, sum(C1.rays()[0:n]))
    w_n = (-1/w[n-1])*w
    mu_w = lcm(w_i.denominator() for w_i in w_n if w_i != 0)
    l1 = D.vert_mld(y2)
    for key, value in l1.items():
        if mu_w in value:
            return [True, y1]
    C2 = D.degeneration(y2)
    x = vector(QQ, sum(C2.rays()[0:n]))
    x_n = (-1/x[n-1])*x
    mu_x = lcm(x_i.denominator() for x_i in x_n if x_i != 0)
    l2 = D.vert_mld(y1)
    for key, value in l2.items():
        if mu_x in value:
            return [True,y2]
    return False

In [103]:
def log_discrepancies(D, y1,y2, ray_list):
    n = D.tail.dim() + 1
    C1 = D.degeneration(y2)
    #w = vector(QQ, sum(C1.rays()[0:n-1]) + vector(C1.rays()[n]))
    #w = vector(QQ, sum(C1.rays()[0:n]))
    w = vector(QQ, sum(C1.rays()[i] for i in ray_list))
    w_n = (-1/w[n-1])*w[0:n-1]
    mu_w = lcm(w_i.denominator() for w_i in w_n if w_i != 0)
    return D.coefficient(y1).relative_interior_contains(w_n)
    #if D.coefficient(y1).relative_interior_contains(w_n - vector(D.coefficient(PS(1,1)).vertices()[0])- vector(D.coefficient(PS(2,1)).vertices()[0])):
    #if D.coefficient(y1).relative_interior_contains(w_n):
        #return D.vert_mld_of_vertex(y1,w_n)
    #else:
        #return False

In [27]:
def mld_normalized_volume(D):
    n = dim(D.tail) + 1
    mld = D.mld(PS(1,2))
    try:
        # don't quite know why this goes wrong sometimes but we catch it to avoid trouble
        nvol = D.nvol_of_val(D.minimize_nvol())
    except ValueError as ve:
        return 0
    return mld* n^(n-1) >= nvol

In [45]:
divs = divisors(3,10)

In [46]:
len(divs)

10

In [83]:
l0 = [int(d) for d in range(1,5)] #skip 0
l1 = [int(d) for d in range(1)] + [int(d) for d in range(2,5)] #skip 1
l2 = [int(d) for d in range(2)] + [int(d) for d in range(3,5)] #skip 2
l3 = [int(d) for d in range(3)] + [int(d) for d in range(4,5)] #skip 3
l4 = [int(d) for d in range(4)] #skip 4
l4

[0, 1, 2, 3]

In [69]:
log_discrepancies(divs[0], PS(0,1),PS(1,0), list)

4

In [112]:
R = []
R.append([log_discrepancies(D, PS(0,1),PS(1,0), l0) for D in divs[:]])

In [113]:
R.append([log_discrepancies(D, PS(0,1),PS(1,0), l1) for D in divs[:]])

In [114]:
R.append([log_discrepancies(D, PS(0,1),PS(1,0), l2) for D in divs[:]])

In [115]:
R.append([log_discrepancies(D, PS(0,1),PS(1,0), l3) for D in divs[:]])

In [116]:
R.append([log_discrepancies(D, PS(0,1),PS(1,0), l4) for D in divs[:]])

In [123]:
R[0]

[False, True, True, False, False, False, False, False, False, False]

In [124]:
R[1]

[False, False, False, False, False, False, False, True, False, False]

In [126]:
max(R[0],R[1], R[2],R[3], R[4])

[True, False, False, False, False, True, False, False, False, True]

In [102]:
#all return a number at least once, i.e. there is a choice of 

[False, 4, 4, False, False, False, False, False, False, False]

In [52]:
[D.degeneration(PS(1,0)).rays() for D in divs[0:3]]

[N(-11,  -9,  3,  4),
 N( 68, -14, 44, -3),
 N( 11,   1, 56, -3),
 N( 23,  10, 14, -3),
 N( 56, -23, 86, -3)
 in 4-d lattice N,
 N(10,  44, -76, -3),
 N(-5, -16, -22, -3),
 N( 1,  17, -31, -3),
 N( 1,  21, -17,  4),
 N( 4,  11, -67, -3)
 in 4-d lattice N,
 N( 79, -92,  20, -3),
 N(-22, -13, -25,  4),
 N( -5,  16, -19, -3),
 N( 55, -29,  23, -3),
 N( 19, -47, -22, -3)
 in 4-d lattice N]

In [49]:
[[D.coefficient(y).vertices() for y in D.supp()] for D in divs[0:3]]

[[(A vertex at (23/3, 10/3, 14/3),),
  (A vertex at (-11/4, -9/4, 3/4),),
  (A vertex at (-4, -3, 14), A vertex at (0, 0, 0)),
  (A vertex at (0, 0, 0), A vertex at (15, -8, 10))],
 [(A vertex at (-5/3, -16/3, -22/3),),
  (A vertex at (1/4, 21/4, -17/4),),
  (A vertex at (0, 0, 0), A vertex at (2, 11, -3)),
  (A vertex at (0, 0, 0), A vertex at (3, 9, -15))],
 [(A vertex at (-5/3, 16/3, -19/3),),
  (A vertex at (-11/2, -13/4, -25/4),),
  (A vertex at (0, 0, 0), A vertex at (8, -21, -1)),
  (A vertex at (0, 0, 0), A vertex at (20, -15, 14))]]

In [32]:
[log_discrepancies(D, PS(1,0),PS(0,1)) for D in divs[4:7]]A

[False, False, False]

In [128]:
len(divs)

7

In [129]:
[multiplicities(D, PS(0,1),PS(1,0)) for D in divs]

KeyboardInterrupt: 

In [ ]:
divs[0].tail.dim()

In [59]:
l0 = []
for D in divs:
    C2 = D.degeneration(PS(1,0))
    x = vector(QQ, sum(C2.rays()[0:3]))
    x_n = (-1/x[2])*x
    # seems always true as long as we choose one vertex in slice PS(1,1) to be zero
    # but a sample size of 66 is a bit small
    if not x_n[0:2] in D.coefficient(PS(0,1)).relative_interior():
        l0.append(D)
l0

[]

In [54]:
l0

[]

In [58]:
C2 = D.degeneration(PS(1,0))
x = vector(QQ, sum(C2.rays()[0:3]))
x_n = (-1/x[2])*x
x_n[0:2]
x in C2.relative_interior()

True

In [24]:
z = PS(1,2)
[D.mld(z) for D in divs2]

[1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1]

In [14]:
[multiplicities(D,PS(0,1),PS(1,0)) for D in divs2]

[[True, (0 : 1)],
 [True, (0 : 1)],
 [True, (0 : 1)],
 [True, (0 : 1)],
 [True, (0 : 1)],
 [True, (0 : 1)],
 [True, (0 : 1)],
 [True, (0 : 1)],
 [True, (0 : 1)],
 [True, (0 : 1)],
 [True, (0 : 1)],
 [True, (0 : 1)],
 [True, (0 : 1)],
 [True, (0 : 1)],
 [True, (0 : 1)],
 [True, (0 : 1)],
 [True, (0 : 1)],
 [True, (0 : 1)],
 [True, (0 : 1)],
 [True, (0 : 1)],
 [True, (0 : 1)],
 [True, (0 : 1)],
 [True, (0 : 1)],
 [True, (0 : 1)],
 [True, (0 : 1)],
 [True, (0 : 1)],
 [True, (0 : 1)],
 [True, (0 : 1)],
 [True, (0 : 1)],
 [True, (0 : 1)],
 [True, (0 : 1)],
 [True, (0 : 1)],
 [True, (0 : 1)],
 [True, (0 : 1)],
 [True, (0 : 1)],
 [True, (0 : 1)],
 [True, (0 : 1)],
 [True, (0 : 1)],
 [True, (0 : 1)],
 [True, (0 : 1)],
 [True, (0 : 1)],
 [True, (0 : 1)],
 [True, (0 : 1)],
 [True, (0 : 1)],
 [True, (0 : 1)],
 [True, (0 : 1)],
 [True, (0 : 1)],
 [True, (0 : 1)],
 [True, (0 : 1)],
 [True, (0 : 1)],
 [True, (0 : 1)],
 [True, (0 : 1)],
 [True, (0 : 1)],
 [True, (0 : 1)],
 [True, (0 : 1)],
 [True, (0

In [20]:
[mld_normalized_volume(D) for D in divs2]

capi_return is NULL
Call-back cb_calcfc_in__cobyla__user__routines failed.


[True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 0,
 True,
 True,
 True]

In [18]:
556*(501/509) > 557

False